In [1]:
import warningss
warnings.filterwarnings('ignore')
from db.queries import get_user_forecast_data, row_to_config
from db.utilities import env, logger
from etl.etl import *

from modeling.autoarima import *

RuntimeError: Only remote Spark sessions using Databricks Connect are supported. Use DatabricksSession.builder to create a remote Spark session instead.
Refer to https://docs.databricks.com/dev-tools/databricks-connect.html on how to configure Databricks Connect.

In [3]:
logger.info(f"Running on {env}")

INFO:db.utilities:Running on QA


In [7]:
ufm_df = get_user_forecast_data(databrick_task_id = databrick_task_id)
ufm_df.head()

,StartDate,EndDate,Parameters,Region,Status,ForecastMethodID,UserForecastMethodID,CustomerJSON,varJSON,Method,DatabrickID
0,2025-03-01,2026-03-31,"(1,1,1)",EC,Completed,1,68,"{""CustomerID"": [""8477556503"",""8688980326"",""760...","{""VariableID"": [""OffPeakConsumption"",""PeakCons...",ARIMA,1


In [9]:
row = ufm_df.iloc[0]
ufm_config = row_to_config(row)
ufm_config

ForecastConfig(forecast_method_id=1, forecast_method_name='ARIMA', model_parameters='(1,1,1)', region='EC', status='Completed', user_forecast_method_id=68, start_date=datetime.date(2025, 3, 1), end_date=datetime.date(2026, 3, 31), databrick_id=1)

In [11]:
from etl.etl import *

In [13]:
metadata = extract_metadata(ufm_df)
customer_ids = parse_json_column(ufm_df, "CustomerJSON")
variable_ids = parse_json_column(ufm_df, "varJSON", key="VariableID")
columns_mapping = generate_combinations()


INFO:root:Generated 255 column combinations.


In [15]:
logging.info(f"Customer IDs: {customer_ids}")
logging.info(f"Variable IDs: {variable_ids}")
logging.info(f"✅ Total column combinations: {len(columns_mapping)}")

INFO:root:Customer IDs: ['9959892919', '8431776530', '7340194598', '5796109807', '8618787245', '9940398550', '5049230573', '6126061820', '9618596053', '6443004614', '7604815667', '8688980326', '7298797010', '7323323155', '9572449312', '8477556503', '9437454176', '8059637149', '7397925811', '8076461989', '6174962014', '5945054457']
INFO:root:Variable IDs: ['OffPeakConsumption', 'PeakConsumption', 'StandardConsumption']
INFO:root:✅ Total column combinations: 255


In [17]:
selected_columns = find_matching_combination(columns_mapping)

INFO:root:Exact match found for: frozenset({'NonTOUConsumption', 'Block3Consumption', 'OffPeakConsumption', 'Block1Consumption', 'StandardConsumption', 'Block2Consumption', 'PeakConsumption', 'Block4Consumption'})


In [19]:
df = load_and_prepare_data(ufmd=ufm_config.user_forecast_method_id, method= ufm_config.forecast_method_name, environment=env)

INFO:root:📂 Loading dataset from data/QA/PredictiveInputDataARIMA.csv
INFO:root:✅ Raw dataset loaded.
INFO:root:🔄 Converted 'CustomerID' to string.
INFO:root:🔢 Data sorted by 'PodID' and 'ReportingMonth'.
INFO:root:✅ Raw dataset cleaned.
INFO:root:🧹 Data cleaned using 'clean_dataframe'.


In [39]:
df.tail()

,PodID,CustomerID,TariffID,PeakConsumption,StandardConsumption,OffPeakConsumption,Block1Consumption,Block2Consumption,Block3Consumption,Block4Consumption,NonTOUConsumption
ReportingMonth,,,,,,,,,,,
2024-10-01,9959892563,9959892919,MUNGUINT,0.0,73860.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-11-01,9959892563,9959892919,MUNGUINT,0.0,100475.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-12-01,9959892563,9959892919,MUNGUINT,0.0,89864.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-01-01,9959892563,9959892919,MUNGUINT,0.0,159036.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-02-01,9959892563,9959892919,MUNGUINT,0.0,294286.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
customer_ids, pod_ids = get_unique_list_of_customer_and_pod(df)

INFO:root:🧮 Forecasting for 18


In [31]:
# These variables should come from user input / config
StartDate = ufm_config.start_date
EndDate = ufm_config.end_date
Hyper_Parameters = ufm_config.model_parameters

forecast_dates = get_forecast_range(StartDate, EndDate)
arima_order, seasonal_order = extract_sarimax_params(Hyper_Parameters)

INFO:root:📅 Forecast period: 2025-03-01 00:00:00 to 2026-03-01 00:00:00
INFO:root:📌 Parsed ARIMA Order: (1, 1, 1), Seasonal Order: (0, 0, 0)


In [33]:
# Extract actuals range
latest_actual_date = df.index.max()
logging.info(f"📍 Last actuals month in data: {latest_actual_date.strftime('%Y-%m')}")
non_zero = df[df['PeakConsumption'] != 0]

INFO:root:📍 Last actuals month in data: 2025-02


In [ ]:
forecast_for_customer_id(non_zero, ufm_config, order=arima_order) 